# iPhone case (Neural Network training)

*Sofia Shchipinskaya*

### Short plan of training part

1. Transforming the data
2. Installing ResNet18
3. Building the model (defining parameters, ect.)
4. Running the model
5. Converting the model
6. Checking the model performance

In [15]:
# importing libraries
import os
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import resnet50, inception_v3, vgg16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.optimizers import Adam
import keras
from keras import backend as K
from keras.models import load_model
import tensorflow as tf
from sklearn import metrics
from keras.callbacks import LearningRateScheduler
import math
from classification_models.resnet import ResNet18
from keras.models import load_model
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.contrib import lite
from sklearn.metrics import average_precision_score

### Transformation of the data

In [16]:
# creating random transformation
val_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rescale=1./255,
        fill_mode='nearest')

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rescale=1./255,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(
        rescale=1./255,
        fill_mode='nearest')

In [17]:
# creating data loaders
val_generator = val_datagen.flow_from_directory(
        'full dataset/val',  
        target_size=(224, 224),  
        batch_size=64,
        class_mode='binary')  

train_generator = val_datagen.flow_from_directory(
        'full dataset/train',  
        target_size=(224, 224), 
        batch_size=64,
        class_mode='binary')  

test_generator = val_datagen.flow_from_directory(
        'full dataset/test',  
        target_size=(224, 224), 
        batch_size=64,
        shuffle=False,
        class_mode='binary')  

Found 2000 images belonging to 2 classes.
Found 48684 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [18]:
train_generator.class_indices

{'An': 0, 'Ip': 1}

### Installing the neural network and running it

In [19]:
# installing ResNet18
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-vjjjooc3
  Stored in directory: /tmp/pip-ephem-wheel-cache-p2wp9oq5/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [20]:
# building the model
model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=True)
model.layers.pop()
last = model.layers[-1].output
x = Dense(2, activation="softmax")(last)
model = Model(model.input, x)

#### comment
The next step is to define an optimizer. I tried Adam and SGD optimizers, however, in this version of the model i decided to use Adam optimizer.

In [8]:
# defining optimizer
# sgd = optimizers.SGD(lr=0.001, decay=0.0, momentum=0.0, nesterov=False)

In [21]:
# defining hyperparameters for tunning
model.compile(loss='sparse_categorical_crossentropy',
#               optimizer=sgd,
              optimizer='adam',
              metrics=['accuracy'])

# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer=sgd,
# #               optimizer=Adam(lr=0.001),
#               metrics=['acc',f1])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
# defining batch sizes
batch_size=64

In [23]:
# defining sample sizes 
nb_samples_train = len(train_generator.filenames)
nb_samples_val= len(val_generator.filenames)
nb_samples_test = len(test_generator.filenames)

#### comment
I decided to use learing rate schedule, so there are some steps to set it.

In [24]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate=0.001
    drop=0.5
    epochs_drop=10
    lrate=initial_lrate*math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [25]:
# learning rate callback
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

#### comment
The next part is the part of model training. As you can see, I chose to train 80 epochs. In the previous version of training part I set 100 epochs and trained them all (you can see that file in my repo). Due to techical reasons (poor Internet connection and some problems with Google Cloud GPU) I had no time to train the model (previous model took 24h to train with epoch=100 and batch_size=16). In this version I chose epoch=80 and batch_size=64.

In [26]:
# training the model
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_samples_train/batch_size,
        epochs=80,
        validation_data=val_generator,
        validation_steps=nb_samples_val/batch_size,
        callbacks=callbacks_list)

Epoch 1/80
761/760 [==============================] - 645s 847ms/step - loss: 0.5369 - acc: 0.7555 - val_loss: 0.5357 - val_acc: 0.7525
Epoch 2/80
761/760 [==============================] - 638s 839ms/step - loss: 0.3349 - acc: 0.8470 - val_loss: 0.5139 - val_acc: 0.8015
Epoch 3/80
761/760 [==============================] - 640s 840ms/step - loss: 0.2980 - acc: 0.8670 - val_loss: 0.3536 - val_acc: 0.8345
Epoch 4/80
761/760 [==============================] - 637s 838ms/step - loss: 0.2728 - acc: 0.8770 - val_loss: 0.4252 - val_acc: 0.8245
Epoch 5/80
761/760 [==============================] - 637s 837ms/step - loss: 0.2504 - acc: 0.8892 - val_loss: 0.2892 - val_acc: 0.8705
Epoch 6/80
761/760 [==============================] - 640s 841ms/step - loss: 0.2418 - acc: 0.8947 - val_loss: 0.3044 - val_acc: 0.8570
Epoch 7/80
761/760 [==============================] - 639s 840ms/step - loss: 0.2290 - acc: 0.9002 - val_loss: 0.3173 - val_acc: 0.8555
Epoch 8/80
761/760 [============================

Epoch 62/80
761/760 [==============================] - 639s 840ms/step - loss: 0.0258 - acc: 0.9904 - val_loss: 0.1521 - val_acc: 0.9595
Epoch 63/80
761/760 [==============================] - 640s 842ms/step - loss: 0.0273 - acc: 0.9896 - val_loss: 0.1680 - val_acc: 0.9480
Epoch 64/80
761/760 [==============================] - 639s 840ms/step - loss: 0.0267 - acc: 0.9904 - val_loss: 0.1962 - val_acc: 0.9525
Epoch 65/80
761/760 [==============================] - 639s 839ms/step - loss: 0.0266 - acc: 0.9902 - val_loss: 0.1716 - val_acc: 0.9555
Epoch 66/80
761/760 [==============================] - 635s 834ms/step - loss: 0.0249 - acc: 0.9907 - val_loss: 0.1730 - val_acc: 0.9560
Epoch 67/80
761/760 [==============================] - 640s 841ms/step - loss: 0.0248 - acc: 0.9905 - val_loss: 0.1900 - val_acc: 0.9430
Epoch 68/80
761/760 [==============================] - 639s 839ms/step - loss: 0.0271 - acc: 0.9900 - val_loss: 0.1669 - val_acc: 0.9560
Epoch 69/80
761/760 [====================

In [27]:
# saving the model and weights
model.save('my_model2.h5')
model.save_weights('my_weights2.h5')

#### comment
As GitHub doesn't allow to upload files of more than 100Mb, I converted the model to decrease the size.

In [28]:
# Converting keras model to tflite model
converter = lite.TFLiteConverter.from_keras_model_file('my_model2.h5')
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 100 variables.
INFO:tensorflow:Converted 100 variables to const ops.


46791100

### Checking perfomance

In [29]:
#create function for loading and rescaling images
def load_image(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    
    img_tensor = np.expand_dims(img_tensor, axis=0)         
    img_tensor /= 255.
    
    return img_tensor

In [30]:
# loading converted model, setting input and output
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [31]:
# storing predictions
pred = []
for subdir, dirs, files in os.walk('full dataset/test'):
    for file in files:
        input_data = load_image(os.path.join(subdir, file))
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        pred.append(output_data[0])

In [32]:
# calculating avarage percision
average_precision = average_precision_score(test_generator.classes, [x[1] for x in pred])
print('Average precision is: {0:0.10f}'.format(average_precision))

Average precision is: 0.9960567357


#### comment
So, I trained here 80 epochs, the average precision is quite high (99.6%)